<a href="https://colab.research.google.com/github/rocket0l4/kag-cont/blob/main/Insurance_reg(kag_compt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [2]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
!pip uninstall xgboost scikit-learn -y

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


In [4]:
!pip install xgboost scikit-learn

  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.0 MB/s eta 0:00:00
Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 4.8 MB/s eta 0:00:00


In [5]:
!pip install scikit-learn==1.2.2

  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [6]:
!pip install opendatasets

In [7]:
import opendatasets as od

In [8]:
od.download("https://www.kaggle.com/competitions/playground-series-s4e12/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: amansingh0000000
Your Kaggle Key: ··········


100%|██████████| 75.5M/75.5M [00:00<00:00, 125MB/s]



Extracting archive ./playground-series-s4e12/playground-series-s4e12.zip to ./playground-series-s4e12


In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error

In [10]:
train_data = pd.read_csv('/content/playground-series-s4e12/train.csv')

In [11]:
cate_columns = ['Education Level','Exercise Frequency','Occupation','Location','Policy Type','Smoking Status','Property Type','Customer Feedback','Gender','Marital Status']

In [12]:
# def prep_data(df,label_encoders=None,is_train=True,latest_date = None):

#     df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'],errors = 'coerce')

#     if is_train:
#         latest_date = df['Policy Start Date'].max()


#     def assign_quarter_interval(policy_date,latest):
#         if pd.isnull(policy_date):
#             return -1
#         diff_years = latest.year - policy_date.year
#         diff_months = latest.month - policy_date.month
#         total_diff_months = diff_years * 12 + diff_months
#         interval = total_diff_months // 3

#         return interval

#     df['Policy_Start_Interval'] = df['Policy Start Date'].apply(lambda x: assign_quarter_interval(x,latest_date))
#     df['Policy_Start_Interval'] = df['Policy_Start_Interval'].astype(int)


#     df = df.drop('Policy Start Date',axis=1)

#     if is_train:
#         label_encoders = {}
#         for column in cate_columns:
#             if df[column].isnull().any():
#                 df[column] = df[column].fillna('Missing')

#             encoder = LabelEncoder()
#             df[column] = encoder.fit_transform(df[column])
#             label_encoders[column] = encoder

#         return df,label_encoders, latest_date

#     else:
#         for column in cate_columns:
#             if df[column].isnull().any():
#                 df[column] = df[column].fillna('Missing')


#             if label_encoders and column in label_encoders:
#                 encoder = label_encoders[column]

#                 df[column] = df[column].map(lambda s: '<unknown>' if s not in encoder.classes_ else s)

#                 if '<unknown>' not in encoder.classes_:
#                     encoder.classes_ = np.append(encoder.classes_, '<unknown>')

#                 else:
#                     raise ValueError(f'Label encoder for {column} not provided')
#         return df

In [13]:
def prep_data(df,label_encoders=None,is_train=True,latest_date = None):
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'],errors = 'coerce')

    if is_train:
        latest_date = df['Policy Start Date'].max()


    def assign_quarter_interval(policy_date,latest):
        if pd.isnull(policy_date):
            return -1
        diff_years = latest.year - policy_date.year
        diff_months = latest.month - policy_date.month
        total_diff_months = diff_years * 12 + diff_months
        interval = total_diff_months // 3

        return interval

    df['Policy_Start_Interval'] = df['Policy Start Date'].apply(lambda x: assign_quarter_interval(x,latest_date))
    df['Policy_Start_Interval'] = df['Policy_Start_Interval'].astype(int)


    df = df.drop('Policy Start Date',axis=1)

    if is_train:
        label_encoders = {}
        for column in cate_columns:
            if df[column].isnull().any():
                df[column] = df[column].fillna('Missing')

            encoder = LabelEncoder()
            df[column] = encoder.fit_transform(df[column])
            label_encoders[column] = encoder

        return df,label_encoders, latest_date

    else:
        for column in cate_columns:
            if df[column].isnull().any():
                df[column] = df[column].fillna('Missing')
            # The change is here to handle the unknown categories numerically, by giving them value of -1
            if label_encoders and column in label_encoders:
                encoder = label_encoders[column]

                # Replace unknown categories with -1 instead of appending '<unknown>' to encoder classes
                df[column] = df[column].map(lambda s: -1 if s not in encoder.classes_ else encoder.transform([s])[0])
                df[column] = df[column].astype(int)  # Ensure the column type is int
                # Convert all the object type column into integer type

        return df

In [14]:
train_data,label_encoders, latest_date = prep_data(train_data,is_train = True)

X_train_full = train_data.drop(['id','Premium Amount'],axis=1)
Y_train_full = train_data['Premium Amount']

In [15]:
test_data = pd.read_csv('/content/playground-series-s4e12/test.csv')

In [16]:
test_ids = test_data['id'].copy()

In [17]:
test_data_processed = prep_data(test_data,label_encoders=label_encoders,is_train=False,latest_date=latest_date)


X_test = test_data_processed.drop(['id'],axis=1,errors='ignore')

In [18]:
Y_train_full_log = np.log1p(Y_train_full)

In [19]:
model = xgb.XGBRegressor(
    objective = 'reg:squarederror',
    random_state = 42,
    n_estimators = 1000,
    learning_rate = 0.05,
    max_depth = 8,
    subsample = 0.8,
    colsample_bytree = 0.8,
    early_stopping_rounds = 50,
    eval_metrics = 'rmse',
    verbosity = 1
)

In [20]:
X_train_part,X_val_part, y_train_part,y_val_part = train_test_split(X_train_full,Y_train_full_log,test_size=0.2,random_state=42)

In [21]:
model.fit(
    X_train_part,y_train_part,
    eval_set = [(X_val_part,y_val_part)],
    verbose=100
)

[0]	validation_0-rmse:1.09190


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:31:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "eval_metrics" } are not used.

  warnings.warn(smsg, UserWarning)


[100]	validation_0-rmse:1.04757
[200]	validation_0-rmse:1.04745
[224]	validation_0-rmse:1.04746


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, eval_metrics='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [22]:
optimal_estimators = model.best_iteration
model = xgb.XGBRegressor(
    objective = 'reg:squarederror',
    random_state = 42,
    n_estimators = optimal_estimators,
    learning_rate = 0.05,
    max_depth = 8,
    subsmaple = 0.8,
    colsample_bytree = 0.8,
    eval_metric = 'rmse',
    verbosity = 1
)

In [23]:
model.fit(
    X_train_full,Y_train_full_log,
    verbose =100
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:31:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "subsmaple" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=175, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [24]:
pred_log_test = model.predict(X_test)
pred_test = np.expm1(pred_log_test)

In [25]:
submission = pd.DataFrame({
    'id' : test_ids,
    'Premium Amount' : pred_test
})

In [26]:
print(submission.head())

        id  Premium Amount
0  1200000      743.388550
1  1200001      782.719788
2  1200002      758.397949
3  1200003      808.737854
4  1200004      751.582275


In [27]:
submission.to_csv('submission.csv',index=False)
print("Submission file 'submission.csv' created sucessfully")

Submission file 'submission.csv' created sucessfully
